<div align="center">

# Dirac-3 Continuous

<h4>
  Wesley Dyk<br>
  <small style="font-weight: normal;">
    Senior Quantum Solutions Architect<br>
    Quantum Computing Inc.
  </small>
</h4>

<br>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qci-wdyk/eqc-models-tutorial/blob/main/tutorial03-continuous.ipynb)

</div>

## Encoding

To encode variables in continuous values, Dirac-3 uses photon count in a time bin as a linearly scaled representation of the value for the qudit associated with that time bin. To obtain the qudit values, the photon counts across all qudits are normalized to a total equal to the sum constraint value. Where $R$ is the sum constraint value, $c_i$ is the photon count for time bin $i$, 
$$
x_i = R \left(\frac{c_i}{\sum_j c_j}\right)
$$
The details on how the photon counts can represent this are proprietary. Each qudit is represented by a discrete number of levels. This number of levels subject to the sensitivity of the device, given environmental changes. The minimum number of levels is 200, which directly corresponds to the minimum sensitivity of the device in decibels, approximately 23 dB. The max sensitivity that Dirac-3 can achieve currently is 40 dB. This corresponds to 10,000 levels. 

## Sum Constraint
In the previous tutorial, the `sum_constraint` parameter was introduced.
$$
\sum_i x_i = R
$$

## Distillation
There isn't an error correction scheme needed for this computing paradigm like there is for gate model devices. The EQC does have to mitigate shot noise and the output values could have an error rate of up to $\sqrt{c_i}$. This provides an opportunity for post-processing that we call distillation, done using sampling from distributions based on the state vector ($(c_0, c_1,\ldots,c_{n-1})$) given by the device. Distillation is triggered by requesting a solution precision.

## Supported Polynomial
Dirac-3 has the capability of representing up to fifth-order terms:
$$
E(x)=\sum_i C_i x_i + \sum_i\sum_j J_{ij}x_ix_j + \sum_i\sum_j\sum_k T_{ijk} x_ix_jx_k + \sum_i\sum_j\sum_k\sum_l Q_{ijkl} x_ix_jx_kx_l+ \sum_i\sum_j\sum_k\sum_l\sum_m P_{ijklm} x_ix_jx_kx_lx_m .
$$

## Device Configuration Parameters

Dirac-3 supports three parameters which directly impact the evolution of a solution sample.

- `relaxation_schedule` is a parameter which represents the choice of one of 4 preconfigured parameter settings. 1 is a choice that also includes an artificial limit on the number of iterations. For the options 2, 3 and 4, the higher the number, the higher the probability of finding the ground state in a single sample. Trade-offs between the number of samples requested and relaxation schedule can lead to more efficient sampling.
- `mean_photon_number` sets the average number of photons in every quantum state. Allowed values are floating point numbers between 6.67e-05 and 0.0066666.
- `quantum_fluctuation_coefficient` sets the amount of loss introduced into the system for each loop in the measurement process. Allowed values are integers between 1 and 50.


## Imports

In [1]:
!pip install eqc_models
import os
import numpy as np
from eqc_models.base import PolynomialModel
from eqc_models.solvers import Dirac3ContinuousCloudSolver
try:
    from google.colab import userdata
except ImportError:
    userdata = None


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## API Keys

In [2]:
# Define the API URL and token  for QCI
api_url ="https://api.qci-prod.com"
if userdata is None:
    api_token = "" # replace or use environment variables to configure
else:
    api_token = userdata.get("QCI_TOKEN")
    os.environ["QCI_TOKEN"] = api_token
    os.environ["QCI_API_URL"] = api_url

## Cubic Example
$$
E(x) = -0.75x_1^2-0.25x_2^2-x_3^2 + x_1x_2x_3
$$

In [3]:
R = 2
coefficients = [-0.75, -0.25, -1, 1]
indices = [(0, 1, 1), (0, 2, 2), (0, 3, 3), (1, 2, 3)]

# Create the polynomial model
model = PolynomialModel(coefficients, indices)
model.upper_bound = R*np.ones((3,)) # Upper bounds 2 for 3 variables

In [4]:
solver = Dirac3ContinuousCloudSolver(url=api_url, api_token=api_token)
response = solver.solve(model, sum_constraint=2, relaxation_schedule=1, num_samples=5) # Set the sum constraint to 2 and relaxation schedule to 1, 5 samples.
print(response["results"]["solutions"])
print(response["results"]["energies"])
print(response["results"]["counts"])

2025-08-28 22:13:17 - Dirac allocation balance = 0 s (unmetered)
2025-08-28 22:13:17 - Job submitted: job_id='68b128dd8060c9339796369d'
2025-08-28 22:13:17 - QUEUED
2025-08-28 22:13:19 - RUNNING
2025-08-28 22:13:22 - COMPLETED
2025-08-28 22:13:25 - Dirac allocation balance = 0 s (unmetered)
[[2, 0, 0], [2, 1e-07, 0]]
[-3, -3]
[3, 2]


High `mean_photon_number` : Corresponds to higher-intensity pulses. This forces the system to converge more quickly, but it increases the risk of getting trapped in a sub-optimal (local) minimum.

In [5]:
solver = Dirac3ContinuousCloudSolver(url=api_url, api_token=api_token)
response = solver.solve(model, sum_constraint=2, relaxation_schedule=1, num_samples=10, mean_photon_number=0.0066666)
print(response["results"]["solutions"])
print(response["results"]["energies"])
print(response["results"]["counts"])

2025-08-28 22:13:26 - Dirac allocation balance = 0 s (unmetered)
2025-08-28 22:13:26 - Job submitted: job_id='68b128e68060c9339796369f'
2025-08-28 22:13:26 - QUEUED
2025-08-28 22:13:31 - RUNNING
2025-08-28 22:13:34 - COMPLETED
2025-08-28 22:13:36 - Dirac allocation balance = 0 s (unmetered)
[[2, 0, 0], [2, 1e-07, 1e-07]]
[-3, -3]
[6, 4]


Low `mean_photon_number` : Corresponds to lower-intensity pulses. This allows the solver to better explore the problem's energy landscape to find the true optimal solution, but it may take longer to settle on a final answer.

In [6]:
solver = Dirac3ContinuousCloudSolver(url=api_url, api_token=api_token)
response = solver.solve(model, sum_constraint=2, relaxation_schedule=1, num_samples=5, mean_photon_number=6.67e-05)
print(response["results"]["solutions"])
print(response["results"]["energies"])
print(response["results"]["counts"])

2025-08-28 22:13:38 - Dirac allocation balance = 0 s (unmetered)
2025-08-28 22:13:38 - Job submitted: job_id='68b128f28060c933979636a1'
2025-08-28 22:13:38 - QUEUED
2025-08-28 22:13:40 - RUNNING
2025-08-28 22:13:56 - COMPLETED
2025-08-28 22:13:58 - Dirac allocation balance = 0 s (unmetered)
[[0, 0, 2], [2, 0, 0]]
[-4, -3]
[3, 2]


In [7]:
for sol in response["results"]["solutions"]:
    print(model.polynomial.evaluate(np.array(sol)))

-4.0
-3.0


## Slack Qudit
For most problems, the sum of all variables is not known beforehand. To support this requirement, a "slack" qudit can be used.

This polynomial is minimized with $x_1=0,x_2=0$.

$$
E(x)=x_1+x_2+x_1x_2
$$

In [8]:
coefficients = [1, 1, 1]
indices = [(0, 1), (0, 2), (1, 2)]
model = PolynomialModel(coefficients, indices)
model.upper_bound = np.ones((2,))
model.machine_slacks = 1

Notice the extra value in the solution.

In [9]:
response = solver.solve(model, sum_constraint=1, num_samples=5, relaxation_schedule=1)
print(response["results"]["solutions"])
print(response["results"]["energies"])
print(response["results"]["counts"])

2025-08-28 22:13:59 - Dirac allocation balance = 0 s (unmetered)
2025-08-28 22:13:59 - Job submitted: job_id='68b129088060c933979636a2'
2025-08-28 22:14:00 - QUEUED
2025-08-28 22:14:02 - RUNNING
2025-08-28 22:14:05 - COMPLETED
2025-08-28 22:14:07 - Dirac allocation balance = 0 s (unmetered)
[[0, 0, 1]]
[0]
[5]
